### preprocess GHCND to contain information about SNOW from stations in New York borough

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# downloading 2019 GHCND file
from os.path import getsize
from urllib.request import urlretrieve

output_dir = "../raw_data"
out = f'2019.csv'
url = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2019.csv.gz"
urlretrieve(url, f"{output_dir}/{out}")

print(f"Done downloading {out} to {output_dir} with size {getsize(f'{output_dir}/{out}') / 1073741824:.2f}GB")

Done downloading 2019try.csv to ../raw_data with size 0.14GB


In [9]:
# read the downloaded '.csv.gzip' file into dataframe
import numpy as np

col_dtypes = {'station':np.str, 'date':np.int32, 'element':np.str, 'value':np.int32, 'Mflag':np.str, 'Qflag':np.str, 'Sflag':np.str, 'time': np.float64}
GHCND_df = pd.read_csv("../raw_data/2019.csv", compression={'method':'gzip'}, names=['station','date','element','value','Mflag','Qflag','Sflag','time'], dtype=col_dtypes)
GHCND_df.tail()

C:\Users\amy_w\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


,station,date,element,value,Mflag,Qflag,Sflag,time
34732301,WQW00041606,20191231,TMIN,222,NaN,NaN,S,NaN
34732302,WQW00041606,20191231,TAVG,239,H,NaN,S,NaN
34732303,ZI000067775,20191231,TMAX,334,NaN,NaN,S,NaN
34732304,ZI000067775,20191231,TMIN,154,NaN,NaN,S,NaN
34732305,ZI000067775,20191231,TAVG,247,H,NaN,S,NaN


In [10]:
GHCND_df.dtypes

station     object
date         int32
element     object
value        int32
Mflag       object
Qflag       object
Sflag       object
time       float64
dtype: object

In [11]:
# read the station information file to get station ID for stations within the New York borough range, in order to filter
# GHCND_df to contain data of stations within New York borough.
data = []
f = open('../raw_data/NYstations.csv', 'r')
for line in f.readlines():
    line_split = line.split('  ')
    row = [line_split[0], float(line_split[1]), float(line_split[2])]
    data.append(row)
station_df = pd.DataFrame(data, columns=['station', 'lat', 'lon'])
print(station_df.dtypes, '\n', station_df.tail())

station     object
lat        float64
lon        float64
dtype: object 
          station      lat      lon
837  US1NYYT0005  42.7326 -77.0766
838  US1NYYT0006  42.5110 -76.9155
839  US1NYYT0007  42.5284 -76.9776
840  US1NYYT0008  42.6539 -77.2660
841  US1NYYT0009  42.6042 -77.3007


In [12]:
# extract stations which are in the New York borough range
# range reference: 'Lab2 R tutorial'->'Boundary Plot'
borough_stations = station_df.loc[(station_df['lat']>40.5)&(station_df['lat']<40.9)&(station_df['lon']>-74.3)&(station_df['lon']<-73.7)]['station']
borough_stations

46     US1NYBX0025
319    US1NYKN0003
320    US1NYKN0025
399    US1NYNS0007
421    US1NYNY0074
422    US1NYNY0078
544    US1NYQN0002
545    US1NYQN0026
546    US1NYQN0027
547    US1NYQN0029
548    US1NYQN0033
549    US1NYQN0036
550    US1NYQN0037
551    US1NYRC0001
552    US1NYRC0002
Name: station, dtype: object

In [16]:
# filter GHCND using the station ID in borough_stations
# now we have GHCND_borough_df containing data from stations within New York borough range
GHCND_borough_df = GHCND_df.loc[(GHCND_df['station'].isin(borough_stations))]
GHCND_borough_df.tail()

,station,date,element,value,Mflag,Qflag,Sflag,time
34678284,US1NYQN0002,20191231,PRCP,84,NaN,NaN,N,NaN
34678285,US1NYQN0027,20191231,DAPR,5,NaN,NaN,N,NaN
34678286,US1NYQN0027,20191231,MDPR,193,NaN,NaN,N,NaN
34678287,US1NYRC0001,20191231,PRCP,259,NaN,NaN,N,NaN
34678288,US1NYRC0002,20191231,PRCP,56,NaN,NaN,N,NaN


In [17]:
# further filter the dataframe to contain rows with SNOW element, as SNOW is our interest
# the three flag attributes are not useful in further analysis, and time has many NaN values, 
# so extract only date and value attributes
SNOW_df = GHCND_borough_df.loc[GHCND_borough_df['element']=='SNOW', ['date', 'value']]
print(SNOW_df.shape)
# create and write into csv file
SNOW_df.to_csv('../preprocessed_data/SNOW.csv', index=False)

(1160, 2)


In [13]:
# may keep a record of GHCND_borough_df
#GHCND_borough_df.to_csv('../preprocessed_data/GHCND_borough.csv', index=False)